In [1]:
import transformers
from transformers import XLNetTokenizer, XLNetModel, AdamW, get_linear_schedule_with_warmup
import torch

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.utils import shuffle
from collections import defaultdict
from textwrap import wrap
from pylab import rcParams
import re

from torch import nn, optim
from keras_preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset,RandomSampler,SequentialSampler
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F





In [2]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
# Reading data using pandas
path_to_data = "tweets.csv"
df = pd.read_csv(path_to_data, encoding = 'raw_unicode_escape', engine ='python',header=None)

# Shuffle and Clip data
df = shuffle(df)
df = df[:20000]

# Function to clean text. Remove tagged entities, hyperlinks, emojis
def clean_text(text):
    text = re.sub(r"@[A-Za-z0-9]+", ' ', text)
    text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)
    text = re.sub(r"[^a-zA-z.!?'0-9]", ' ', text)
    text = re.sub('\t', ' ',  text)
    text = re.sub(r" +", ' ', text)
    return text
 
df['tweet'] = df[5].apply(clean_text)

# Function to convert labels to number.
def sentiment2label(sentiment):
    if sentiment == 4 or sentiment == 3:
        return 1
    else :
        return 0

df['sentiment'] = df[0].apply(sentiment2label)

# List of class names.
class_names = ['negative', 'positive']

In [5]:
df['sentiment'].value_counts()

1    10058
0     9942
Name: sentiment, dtype: int64

In [6]:
print(len(df))
df.head(20)

20000


,0,1,2,3,4,5,tweet,sentiment
541200,0,2200003196,Tue Jun 16 18:18:12 PDT 2009,NO_QUERY,LaLaLindsey0609,@chrishasboobs AHHH I HOPE YOUR OK!!!,AHHH I HOPE YOUR OK!!!,0
750,0,1467998485,Mon Apr 06 23:11:14 PDT 2009,NO_QUERY,sexygrneyes,"@misstoriblack cool , i have no tweet apps fo...",cool i have no tweet apps for my razr 2,0
766711,0,2300048954,Tue Jun 23 13:40:11 PDT 2009,NO_QUERY,sammydearr,@TiannaChaos i know just family drama. its la...,i know just family drama. its lame.hey next t...,0
285055,0,1993474027,Mon Jun 01 10:26:07 PDT 2009,NO_QUERY,Lamb_Leanne,School email won't open and I have geography ...,School email won't open and I have geography s...,0
705995,0,2256550904,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,yogicerdito,upper airways problem,upper airways problem,0
379611,0,2052380495,Sat Jun 06 00:32:16 PDT 2009,NO_QUERY,Yengching,Going to miss Pastor's sermon on Faith...,Going to miss Pastor's sermon on Faith...,0
1189018,4,1983449090,Sun May 31 13:10:36 PDT 2009,NO_QUERY,jessig06,on lunch....dj should come eat with me,on lunch....dj should come eat with me,1
667030,0,2245479748,Fri Jun 19 16:11:29 PDT 2009,NO_QUERY,felicityfuller,@piginthepoke oh why are you feeling like that?,oh why are you feeling like that?,0
93541,0,1770705699,Mon May 11 22:01:32 PDT 2009,NO_QUERY,stephiiheyy,gahh noo!peyton needs to live!this is horrible,gahh noo!peyton needs to live!this is horrible,0
1097326,4,1970386589,Sat May 30 03:39:34 PDT 2009,NO_QUERY,wyndwitch,@mrstessyman thank you glad you like it! There...,thank you glad you like it! There is a produc...,1


In [7]:
from transformers import XLNetTokenizer, XLNetModel
PRE_TRAINED_MODEL_NAME = 'xlnet-base-cased'
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

In [8]:
class tweetDataset(Dataset):

    def __init__(self, tweets, targets, tokenizer, max_len):
        self.tweets = tweets
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.tweets)
    
    def __getitem__(self, item):
        tweet = str(self.tweets[item])
        target = self.targets[item]

        encoding = self.tokenizer.encode_plus(
        tweet,
        add_special_tokens=True,
        max_length=self.max_len,
        return_token_type_ids=False,
        pad_to_max_length=False,
        return_attention_mask=True,
        return_tensors='pt',
        )

        input_ids = pad_sequences(encoding['input_ids'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
        input_ids = input_ids.astype(dtype = 'int64')
        input_ids = torch.tensor(input_ids) 

        attention_mask = pad_sequences(encoding['attention_mask'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
        attention_mask = attention_mask.astype(dtype = 'int64')
        attention_mask = torch.tensor(attention_mask)       

        return {
        'tweet': tweet,
        'input_ids': input_ids,
        'attention_mask': attention_mask.flatten(),
        'targets': torch.tensor(target, dtype=torch.long)
        }

In [9]:
df_train, df_test = train_test_split(df, test_size=0.5, random_state=101)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=101)
df_train.shape, df_val.shape, df_test.shape

((10000, 8), (5000, 8), (5000, 8))

In [10]:

def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = tweetDataset(
    tweets=df.tweet.to_numpy(),
    targets=df.sentiment.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )

  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=0
  )

In [11]:
BATCH_SIZE = 4
MAX_LEN = 280
train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

In [12]:
from transformers import XLNetForSequenceClassification
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels = 2)
model = model.to(device)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [13]:
model

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward

In [14]:

EPOCHS = 6
BATCH_SIZE = 4

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
                                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
                                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay':0.0}
]
optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=3e-5)
#optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5)

total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)
print("done")

done


In [15]:
data = next(iter(val_data_loader))
data.keys()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


dict_keys(['tweet', 'input_ids', 'attention_mask', 'targets'])

In [16]:
input_ids = data['input_ids'].to(device)
attention_mask = data['attention_mask'].to(device)
targets = data['targets'].to(device)
print(input_ids.shape)

torch.Size([4, 1, 280])


In [17]:
print(input_ids.reshape(4,280).shape) # batch size x seq length
print(attention_mask.shape) # batch size x seq length

torch.Size([4, 280])
torch.Size([4, 280])


In [18]:
input_ids[0]

tensor([[   94,  4394,    27, 14688,    21,    25,   808,    81, 14625,    94,
          4075,   902,    23,     9,    17,   150,   102,   210,   107,    78,
             4,     3,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [19]:
outputs = model(input_ids.reshape(4,280), token_type_ids=None, attention_mask=attention_mask, labels=targets)
outputs

XLNetForSequenceClassificationOutput(loss=tensor(0.8500, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[-0.0048, -0.2940],
        [ 0.4382,  0.0084],
        [ 0.3111, -0.2074],
        [ 0.4721, -0.5836]], device='cuda:0', grad_fn=<AddmmBackward0>), mems=(tensor([[[-1.3140e-02,  1.9877e-02,  2.9697e-02,  ..., -3.8429e-02,
          -4.6014e-02, -4.5964e-02],
         [-2.3878e-02,  6.2314e-02, -3.0385e-02,  ..., -4.3332e-02,
          -2.2326e-02,  5.3156e-03],
         [-9.3024e-05, -1.4913e-02, -5.6822e-02,  ...,  4.3831e-02,
          -5.7188e-02,  8.7464e-02],
         [ 4.8753e-02, -2.1498e-02,  5.7051e-02,  ..., -3.9360e-03,
           2.7743e-04, -1.5234e-02]],

        [[ 1.6359e-03, -5.5736e-03, -3.7612e-02,  ..., -6.0062e-02,
          -4.8277e-02,  1.6078e-01],
         [-3.6630e-02,  2.7185e-02, -3.4077e-02,  ..., -1.2478e-02,
           3.4914e-02,  3.7377e-02],
         [-4.8154e-02, -1.2569e-02,  4.5263e-02,  ..., -7.7962e-02,
          -4.2495e-02,  1.2

In [20]:
print(type(outputs[0]))
outputs[0].shape

<class 'torch.Tensor'>


torch.Size([])

In [21]:
from sklearn import metrics
def train_epoch(model, data_loader, optimizer, device, scheduler, n_examples):
    model = model.train()
    losses = []
    acc = 0
    counter = 0
  
    for d in data_loader:
        input_ids = d["input_ids"].reshape(4,280).to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)
        
        outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels = targets)
        loss = outputs[0]
        logits = outputs[1]

        # preds = preds.cpu().detach().numpy()
        _, prediction = torch.max(outputs[1], dim=1)
        targets = targets.cpu().detach().numpy()
        prediction = prediction.cpu().detach().numpy()
        accuracy = metrics.accuracy_score(targets, prediction)

        acc += accuracy
        losses.append(loss.item())
        
        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        counter = counter + 1

    return acc / counter, np.mean(losses)

In [22]:

def eval_model(model, data_loader, device, n_examples):
    model = model.eval()
    losses = []
    acc = 0
    counter = 0
  
    with torch.no_grad():
        for d in data_loader:
            #print(d["input_ids"])
            #print(d["input_ids"].shape)
            if str(d["input_ids"].shape) == "torch.Size([4, 1, 280])":
                #print("correctSize")
                input_ids = d["input_ids"].reshape(4,280).to(device)
                attention_mask = d["attention_mask"].to(device)
                targets = d["targets"].to(device)

                outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels = targets)
                loss = outputs[0]
                logits = outputs[1]

                _, prediction = torch.max(outputs[1], dim=1)
                targets = targets.cpu().detach().numpy()
                prediction = prediction.cpu().detach().numpy()
                accuracy = metrics.accuracy_score(targets, prediction)

                acc += accuracy
                losses.append(loss.item())
                counter += 1

    return acc / counter, np.mean(losses)

In [23]:
%%time
history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    train_acc, train_loss = train_epoch(
        model,
        train_data_loader,     
        optimizer, 
        device, 
        scheduler, 
        len(df_train)
    )

    print(f'Train loss {train_loss} Train accuracy {train_acc}')

    val_acc, val_loss = eval_model(
        model,
        val_data_loader, 
        device, 
        len(df_val)
    )

    print(f'Val loss {val_loss} Val accuracy {val_acc}')
    print()

    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)

    if val_acc > best_accuracy:
        torch.save(model.state_dict(), 'xlnet_model.bin')
        best_accuracy = val_acc

Epoch 1/6
----------
Train loss 0.637950627496466 Train accuracy 0.7191
Val loss 0.6406168011412025 Val accuracy 0.7942

Epoch 2/6
----------
Train loss 0.5770258671192452 Train accuracy 0.832
Val loss 0.854988156190142 Val accuracy 0.7962

Epoch 3/6
----------
Train loss 0.44823989232298916 Train accuracy 0.897
Val loss 0.9201294507205486 Val accuracy 0.8034

Epoch 4/6
----------
Train loss 0.3470611649279948 Train accuracy 0.9283
Val loss 0.9745067946892232 Val accuracy 0.8016

Epoch 5/6
----------
Train loss 0.22481588021258359 Train accuracy 0.9581
Val loss 1.1091667633758857 Val accuracy 0.808

Epoch 6/6
----------
Train loss 0.1699228408474941 Train accuracy 0.97
Val loss 1.160379149915278 Val accuracy 0.809

CPU times: total: 2h 33min 49s
Wall time: 41min 13s


Using the trained Model

In [31]:
model.load_state_dict(torch.load('xlnet_model.bin'))

<All keys matched successfully>

In [32]:
model = model.to(device)
test_acc, test_loss = eval_model(
  model,
  test_data_loader,
  device,
  len(df_test)
)

print('Test Accuracy :', test_acc)
print('Test Loss :', test_loss)

Test Accuracy : 0.8048
Test Loss : 1.1765491367706098


In [33]:
def get_predictions(model, data_loader):
    model = model.eval()
    
    tweets = []
    predictions = []
    prediction_probs = []
    real_values = []

    with torch.no_grad():
        for d in data_loader:

            tweets = d["tweet"]
            input_ids = d["input_ids"].reshape(4,280).to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)
            
            outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels = targets)

            loss = outputs[0]
            logits = outputs[1]
            
            _, preds = torch.max(outputs[1], dim=1)

            probs = F.softmax(outputs[1], dim=1)

            tweets.extend(tweets)
            predictions.extend(preds)
            prediction_probs.extend(probs)
            real_values.extend(targets)

    predictions = torch.stack(predictions).cpu()
    prediction_probs = torch.stack(prediction_probs).cpu()
    real_values = torch.stack(real_values).cpu()
    return tweets, predictions, prediction_probs, real_values

In [34]:
y_tweet, y_pred, y_pred_probs, y_test = get_predictions(
  model,
  test_data_loader
)

In [35]:
print(classification_report(y_test, y_pred, target_names=class_names))


              precision    recall  f1-score   support

    negative       0.80      0.80      0.80      2492
    positive       0.81      0.81      0.81      2508

    accuracy                           0.80      5000
   macro avg       0.80      0.80      0.80      5000
weighted avg       0.80      0.80      0.80      5000



In [36]:

def predict_sentiment(text):
    tweet = text

    encoded_tweet = tokenizer.encode_plus(
    tweet,
    max_length=MAX_LEN,
    add_special_tokens=True,
    return_token_type_ids=False,
    pad_to_max_length=False,
    return_attention_mask=True,
    return_tensors='pt',
    )

    input_ids = pad_sequences(encoded_tweet['input_ids'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
    input_ids = input_ids.astype(dtype = 'int64')
    input_ids = torch.tensor(input_ids) 

    attention_mask = pad_sequences(encoded_tweet['attention_mask'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
    attention_mask = attention_mask.astype(dtype = 'int64')
    attention_mask = torch.tensor(attention_mask) 

    input_ids = input_ids.reshape(1,280).to(device)
    attention_mask = attention_mask.to(device)

    outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    outputs = outputs[0][0].cpu().detach()

    probs = F.softmax(outputs, dim=-1).cpu().detach().numpy().tolist()
    _, prediction = torch.max(outputs, dim =-1)

    print("Positive score:", probs[1])
    print("Negative score:", probs[0])
    print(f'tweet: {tweet}')
    print(f'Sentiment  : {class_names[prediction]}')

In [51]:
text = "Bitcoin is so in right now"
predict_sentiment(text)

Positive score: 0.996258020401001
Negative score: 0.003741990076377988
tweet: Bitcoin is so in right now
Sentiment  : positive


In [60]:
# Reading data using pandas
path_to_data = "Bitcoin_tweets.csv"
dfBitcoin = pd.read_csv(path_to_data, engine ='python')

# Shuffle and Clip data
dfBitcoin = shuffle(dfBitcoin)
dfBitcoin = dfBitcoin[:10000]



In [70]:
# Function to clean text. Remove tagged entities, hyperlinks, emojis
def clean_text(text):
    text = re.sub(r"@[A-Za-z0-9]+", ' ', text)
    text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)
    text = re.sub(r"[^a-zA-z.!?'0-9]", ' ', text)
    text = re.sub('\t', ' ',  text)
    text = re.sub(r" +", ' ', text)
    return text
 
dfBitcoin['text'] = dfBitcoin['text'].apply(clean_text)


In [97]:
i = 1
import time
for text in dfBitcoin['text']:
    if i <100 :
        predict_sentiment(text)
        print("-------------------------------------\n")
        i = i+1

Positive score: 0.9988799691200256
Negative score: 0.0011199824512004852
tweet: @PeterSchiff @SpencerKSchiff Liking this for @SpencerKSchiff and not for backwards ideals #bitcoin Happy birthday Spence!!
Sentiment  : positive
-------------------------------------

Positive score: 0.0011703784111887217
Negative score: 0.9988296627998352
tweet: #BTC Derivs Sheet (Tweet every 15 min.)
CME Delayed (10 min)
GS Delayed (15 min) and OI = SHARES OUT https://t.co/LGqwZu61nK
Sentiment  : negative
-------------------------------------

Positive score: 0.9988633394241333
Negative score: 0.0011367147089913487
tweet: Bitget exchange: Extensive Guide &amp; Review 

#Cardano #cardanofeed #ADA #crypto #cardanocommunity #bitcoin #CoinMarketCap #blockchain #cryptocurrency #cryptonews #btc $ADA
https://t.co/SQ9BOnvkhv
Sentiment  : positive
-------------------------------------

Positive score: 0.9974307417869568
Negative score: 0.0025692617055028677
tweet: I was terrible Tired today couldn’t even make it t

Positive score: 0.9976035952568054
Negative score: 0.002396386582404375
tweet: Don't wait to buy $poodl. Just buy #poodl and wait.
We are #poodl We are money$$$
#poodlparty #poodltoken https://t.co/auepx1zpOK
#hodlthepoodl #bnb #btc #bsc
Sentiment  : positive
-------------------------------------

Positive score: 0.9931064248085022
Negative score: 0.006893625948578119
tweet: strategy: 5010HL1h atr20d: 2172.51

21 Mar 2022 22:00:03 UTC
🔄 'None' 07:00:03 JST
--- 42429.5  long_entry_trigger
 &gt;  41310.0  last_price $BTC/USD
--- 40421.0  short_entry_trigger

#BTC/USD perpetual contract on #BitMEX
Sentiment  : positive
-------------------------------------

Positive score: 0.004244736395776272
Negative score: 0.9957553148269653
tweet: This might be a stop hunt above previous high and into weekly resistance here. Testing a short now. $btc #btc #bitcoin not financial advice loooool https://t.co/m2JBKTPX40
Sentiment  : negative
-------------------------------------

Positive score: 0.0011582

Positive score: 0.005418220069259405
Negative score: 0.9945817589759827
tweet: #inflation = lose
#bitcoin = freedom, success…
Only 21 000 000  #btc 🚀 limited 

 @saylor @elonmusk @cz_binance @BitcoinMagazine
Sentiment  : negative
-------------------------------------

Positive score: 0.05140131339430809
Negative score: 0.9485986828804016
tweet: 🚀 🚀2,652 #BTC (109,420,904 USD) move from unknown wallet to unknown wallet
 
Date : 2022-04-14 18:02:08 (GMT 0)
Blockchain : #BITCOIN
Symbol : #BTC
 
Detail: https://t.co/jWNcfcoSNY
Sentiment  : negative
-------------------------------------

Positive score: 0.9988707900047302
Negative score: 0.0011292352573946118
tweet: Sign up on the best crypto gambling website in the world! You will get a daily, weekly and monthly bonus!

#casino #slots #crypto #betting #eth #btc #gambling #dice

https://t.co/g3f3I7rEmD https://t.co/ZKnDee8T7F
Sentiment  : positive
-------------------------------------

Positive score: 0.005776320584118366
Negative score: 0.

Positive score: 0.998748779296875
Negative score: 0.0012512303655967116
tweet: Your Condition is not even as critical as those I've Handled,,

Inbox me, If you still want your Account Back!!!

I'm available 24/7 for all Hacking Services
#Hack #coinbase #Hacking #bitcoin   #bnb  #privacy
Sentiment  : positive
-------------------------------------

Positive score: 0.9988712668418884
Negative score: 0.0011286851949989796
tweet: Follow @shakespearby, will teach you how to position yourself in a bear market, also provide you the latest market, and case studies.

#NFTs #NFTCommunity #Airdrop #NFTGiveaways #Crypto  #BTC #ETH 
JUST DONE THE CASE STUDY OF Azuki: https://t.co/FeCCjmQfHe https://t.co/8L9NDqHjPP
Sentiment  : positive
-------------------------------------

Positive score: 0.0016652601771056652
Negative score: 0.9983347058296204
tweet: @CryptoWhale Hi #BTC I never asked you for anything. But the bills are coming...

Hi McDonald's, where can I apply?
Sentiment  : negative
-----------